# OWC estimator v2
## well locations processed as is without any transformation

Добавить MVA и кроссвалидацию

blind test

In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, date
import matplotlib.pyplot as plt
import math
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pylab
from pylab import rcParams

data_path = '../data/ps_owc/df.xlsx'
df_train = pd.read_excel(data_path, sheet_name='train2')
df_test = pd.read_excel(data_path, sheet_name='test2')

Выборка разбита на тренировочный и тестовый сеты

In [ ]:
df_train

In [ ]:
df_test

В тестовой выборке удаляем имя скважины и целевой (предсказываемый) признак

In [ ]:
x_test = df_test.drop(['Well', 'wct'], axis=1)
y_test = df_test['wct']
x_test

Удаляем имя скважины и целевой (предсказываемый) признак в тренировочной выборке

In [ ]:
x = df_train.drop(['Well', 'wct'], axis=1)
y = df_train['wct']
x

Создаём скейлер для масштабирования данных

обучаем скейлер на тренировочных данных и масштабируем их

масштабируем тестовые данные на обученном скейлере

создаём модель LinearRegression / RandomForestRegressor

тренируем модель

рассчитываем обводнённость по тестовой выборке

рассчитываем обводнённость по тренировочной выборке

оцениваем качество модели: считаем коэффициенты R^2 по тестовым и тренировочным выборкам

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x)

x_test = scaler.transform(df_test.drop(['Well', 'wct'], axis=1))

model = RandomForestRegressor(random_state=42, max_depth=14)
model.fit(x_train, y)

y_pred = model.predict(x_test)

y_pred_train = model.predict(x_train)

r2_train = r2(y, y_pred_train)
print(f'R2 train: {r2_train.round(4)}')

r2_test = r2(y_test, y_pred)
print(f'R2 test: {r2_train.round(4)}')

model

In [ ]:
df_y_test = pd.DataFrame({'Well': df_test['Well'], 
                          'wct predicted, %': y_pred.round(1), 
                          'wct actual, %': y_test.round(1)})
df_y_test

In [ ]:
df_y_train = pd.DataFrame({'Well': df_train['Well'], 
                           'wct predicted, %': y_pred_train.round(1), 
                           'wct actual, %': y.round(1)})
df_y_train

In [ ]:
model.feature_importances_
feature_importances = pd.DataFrame()
feature_importances['importance'] = model.feature_importances_
feature_importances['feature_name'] = x.columns.tolist()
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances

In [ ]:
feature_importances = feature_importances.sort_values(by='importance', ascending=True)
height = feature_importances['importance']
bars = feature_importances['feature_name']
y_pos = np.arange(len(bars))
# Create horizontal bars
plt.barh(y_pos, height)
 # Create names on the y-axis
plt.yticks(y_pos, bars)
plt.show()

In [ ]:
def evaluate_preds(true_values, pred_values):
    print("R2:\t" + str(round(r2(true_values, pred_values), 3)) + "\n" +
          "MAE:\t" + str(round(mae(true_values, pred_values), 0)) + "\n" +
          "MSE:\t" + str(round(mse(true_values, pred_values), 0))) 
    plt.figure(figsize=(6,6))
    sns.scatterplot(x=pred_values, y=true_values, s=20)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('True vs Predicted values')
    plt.ylim(0, 105)
    plt.xlim(0, 105)
    plt.show()

evaluate_preds(y.values.flatten(), y_pred_train.flatten())
evaluate_preds(y_test.values.flatten(), y_pred.flatten())